# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 30 2022 1:30PM,240871,20,SSNA-2173706715,Sartorius Stedim North America,Released
1,Jun 30 2022 1:21PM,240870,20,ALUR020992707,Alumier Labs Inc.,Released
2,Jun 30 2022 1:21PM,240870,20,ALUR663785343,Alumier Labs Inc.,Released
3,Jun 30 2022 1:21PM,240870,20,ALUR118973578,Alumier Labs Inc.,Released
4,Jun 30 2022 1:21PM,240870,20,ALUR638034807,Alumier Labs Inc.,Released
5,Jun 30 2022 1:21PM,240870,20,ALUR268624807,Alumier Labs Inc.,Released
6,Jun 30 2022 1:21PM,240870,20,ALUR606420389,Alumier Labs Inc.,Released
7,Jun 30 2022 1:21PM,240870,20,ALUR965774923,Alumier Labs Inc.,Released
8,Jun 30 2022 1:21PM,240870,20,ALUR611050864,Alumier Labs Inc.,Released
9,Jun 30 2022 1:05PM,240869,10,BPS1974,Beach Patient Assistance,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,240867,Executing,3
36,240868,Released,4
37,240869,Released,2
38,240870,Released,8
39,240871,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240867,NaN,3.0,NaN
240868,NaN,NaN,4.0
240869,NaN,NaN,2.0
240870,NaN,NaN,8.0
240871,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240766,0.0,0.0,1.0
240768,0.0,0.0,1.0
240774,0.0,1.0,0.0
240781,0.0,0.0,1.0
240784,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240766,0,0,1
240768,0,0,1
240774,0,1,0
240781,0,0,1
240784,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240766,0,0,1
1,240768,0,0,1
2,240774,0,1,0
3,240781,0,0,1
4,240784,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240766,,,1
1,240768,,,1
2,240774,,1,
3,240781,,,1
4,240784,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.
9,Jun 30 2022 1:05PM,240869,10,Beach Patient Assistance
11,Jun 30 2022 12:48PM,240868,10,Emerginnova
15,Jun 30 2022 12:46PM,240867,12,Hush Hush
18,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc."
19,Jun 30 2022 12:34PM,240863,10,"Methapharm, Inc."
24,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.
25,Jun 30 2022 12:25PM,240860,10,"Citieffe, Inc."
26,Jun 30 2022 12:19PM,240859,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,,,1
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,,,8
2,Jun 30 2022 1:05PM,240869,10,Beach Patient Assistance,,,2
3,Jun 30 2022 12:48PM,240868,10,Emerginnova,,,4
4,Jun 30 2022 12:46PM,240867,12,Hush Hush,,3,
5,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc.",,,1
6,Jun 30 2022 12:34PM,240863,10,"Methapharm, Inc.",,4,1
7,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,,,1
8,Jun 30 2022 12:25PM,240860,10,"Citieffe, Inc.",,,1
9,Jun 30 2022 12:19PM,240859,20,"ACI Healthcare USA, Inc.",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,1,,
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8,,
2,Jun 30 2022 1:05PM,240869,10,Beach Patient Assistance,2,,
3,Jun 30 2022 12:48PM,240868,10,Emerginnova,4,,
4,Jun 30 2022 12:46PM,240867,12,Hush Hush,,3,
5,Jun 30 2022 12:41PM,240866,10,"Citieffe, Inc.",1,,
6,Jun 30 2022 12:34PM,240863,10,"Methapharm, Inc.",1,4,
7,Jun 30 2022 12:31PM,240861,16,Sartorius Stedim Filters Inc.,1,,
8,Jun 30 2022 12:25PM,240860,10,"Citieffe, Inc.",1,,
9,Jun 30 2022 12:19PM,240859,20,"ACI Healthcare USA, Inc.",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,1,,
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8,,
2,Jun 30 2022 1:05PM,240869,10,Beach Patient Assistance,2,,
3,Jun 30 2022 12:48PM,240868,10,Emerginnova,4,,
4,Jun 30 2022 12:46PM,240867,12,Hush Hush,,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,1.0,NaN,NaN
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8.0,NaN,NaN
2,Jun 30 2022 1:05PM,240869,10,Beach Patient Assistance,2.0,NaN,NaN
3,Jun 30 2022 12:48PM,240868,10,Emerginnova,4.0,NaN,NaN
4,Jun 30 2022 12:46PM,240867,12,Hush Hush,NaN,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 30 2022 1:30PM,240871,20,Sartorius Stedim North America,1.0,0.0,0.0
1,Jun 30 2022 1:21PM,240870,20,Alumier Labs Inc.,8.0,0.0,0.0
2,Jun 30 2022 1:05PM,240869,10,Beach Patient Assistance,2.0,0.0,0.0
3,Jun 30 2022 12:48PM,240868,10,Emerginnova,4.0,0.0,0.0
4,Jun 30 2022 12:46PM,240867,12,Hush Hush,0.0,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2167630,17.0,21.0,0.0
12,240867,0.0,3.0,0.0
15,240847,12.0,17.0,0.0
16,722484,3.0,0.0,0.0
17,240768,1.0,0.0,0.0
18,240813,1.0,0.0,0.0
19,722435,6.0,34.0,18.0
20,1445111,37.0,5.0,0.0
21,1203910,3.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2167630,17.0,21.0,0.0
1,12,240867,0.0,3.0,0.0
2,15,240847,12.0,17.0,0.0
3,16,722484,3.0,0.0,0.0
4,17,240768,1.0,0.0,0.0
5,18,240813,1.0,0.0,0.0
6,19,722435,6.0,34.0,18.0
7,20,1445111,37.0,5.0,0.0
8,21,1203910,3.0,2.0,0.0
9,22,240846,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,17.0,21.0,0.0
1,12,0.0,3.0,0.0
2,15,12.0,17.0,0.0
3,16,3.0,0.0,0.0
4,17,1.0,0.0,0.0
5,18,1.0,0.0,0.0
6,19,6.0,34.0,18.0
7,20,37.0,5.0,0.0
8,21,3.0,2.0,0.0
9,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,17.0
1,12,Released,0.0
2,15,Released,12.0
3,16,Released,3.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
Executing,21.0,3.0,17.0,0.0,0.0,0.0,34.0,5.0,2.0,0.0
Released,17.0,0.0,12.0,3.0,1.0,1.0,6.0,37.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
1,Executing,21.0,3.0,17.0,0.0,0.0,0.0,34.0,5.0,2.0,0.0
2,Released,17.0,0.0,12.0,3.0,1.0,1.0,6.0,37.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0
1,Executing,21.0,3.0,17.0,0.0,0.0,0.0,34.0,5.0,2.0,0.0
2,Released,17.0,0.0,12.0,3.0,1.0,1.0,6.0,37.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()